In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
db = pd.read_csv('tested.csv')

In [3]:
db.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
db = db.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis = 1)

In [5]:
db.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,892,0,3,male,34.5,0,0,7.8292
1,893,1,3,female,47.0,1,0,7.0000
2,894,0,2,male,62.0,0,0,9.6875
3,895,0,3,male,27.0,0,0,8.6625
4,896,1,3,female,22.0,1,1,12.2875


In [6]:
db = db.set_index(['PassengerId'])
db = db.rename(columns={'Survived':'target'},inplace=False)

In [7]:
db.describe(include=['O'])

,Sex
count,418
unique,2
top,male
freq,266


In [8]:
db['Sex_F'] = np.where(db['Sex']=='female', 1, 0)

db['Pclass_1'] = np.where(db['Pclass']== 1, 1, 0)
db['Pclass_2'] = np.where(db['Pclass']== 2, 1, 0)
db['Pclass_3'] = np.where(db['Pclass']== 3, 1, 0)

In [9]:
db = db.drop(['Pclass', 'Sex'], axis=1)

In [10]:
db.head()

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3
PassengerId,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,0,1
893,1,47.0,1,0,7.0000,1,0,0,1
894,0,62.0,0,0,9.6875,0,0,1,0
895,0,27.0,0,0,8.6625,0,0,0,1
896,1,22.0,1,1,12.2875,1,0,0,1


In [11]:
db.isnull().sum()

target       0
Age         86
SibSp        0
Parch        0
Fare         1
Sex_F        0
Pclass_1     0
Pclass_2     0
Pclass_3     0
dtype: int64

In [12]:
db.fillna(0, inplace=True)

In [13]:
db.isnull().sum()

target      0
Age         0
SibSp       0
Parch       0
Fare        0
Sex_F       0
Pclass_1    0
Pclass_2    0
Pclass_3    0
dtype: int64

In [14]:
x_train, x_test, y_train, y_test = train_test_split(db.drop(['target'], axis=1),db['target'],test_size=0.3,random_state=1234)

In [15]:
rndforest = RandomForestClassifier(n_estimators=1000, criterion='gini', max_depth=5)
rndforest.fit(x_train,y_train)

RandomForestClassifier(max_depth=5, n_estimators=1000)

In [16]:
prob = rndforest.predict_proba(db.drop('target',axis=1))[:,1]
clas = rndforest.predict(db.drop('target',axis=1))

In [17]:
db['prob'] = prob
db['class'] = clas
db

,target,Age,SibSp,Parch,Fare,Sex_F,Pclass_1,Pclass_2,Pclass_3,prob,class
PassengerId,,,,,,,,,,,
892,0,34.5,0,0,7.8292,0,0,0,1,0.050086,0
893,1,47.0,1,0,7.0000,1,0,0,1,0.903814,1
894,0,62.0,0,0,9.6875,0,0,1,0,0.048404,0
895,0,27.0,0,0,8.6625,0,0,0,1,0.042525,0
896,1,22.0,1,1,12.2875,1,0,0,1,0.910115,1
...,...,...,...,...,...,...,...,...,...,...,...
1305,0,0.0,0,0,8.0500,0,0,0,1,0.047536,0
1306,1,39.0,0,0,108.9000,1,1,0,0,0.899462,1
1307,0,38.5,0,0,7.2500,0,0,0,1,0.045229,0
